In [1]:
!pip install -r "requirements.txt"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import spacy
import numpy as np
import requests
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup

In [3]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 58.5 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


# Pipeline base

El objetivo es realizar el estudio sobre una gran variedad de juegos, y para ello empezaremos construyendo una pipeline capaz de procesar los datos de un único juego de manera que sea fácilmente utilizable como conjunto de entrenamiento de distintos modelos. Posteriormente la escalabilidad será implementada mediante la iteración de la pipeline sobre distintos juegos.

El primer ejemplo será el videojuego Dota 2.

# API para la obtención de reseñas.

El link asociado a las primeras $n$ reseñas del juego de steam con $id = \text{value}$ en lenguaje español es:
- https://store.steampowered.com/appreviews/value?json=1&num_per_page=n&language=spanish

A partir de esta API podemos obtener fácilmente las reseñas de cualquier juego.

In [4]:
import requests

# URL del endpoint de reseñas
url = 'https://store.steampowered.com/appreviews/570?json=1&num_per_page=100&language=spanish'

# Realiza la petición
response = requests.get(url)
data = response.json() 

# Itera por las reseñas
for review in data['reviews']:
    print("Usuario:", review['author']['steamid'])
    print("Comentario:", review['review'])
    print("¿Recomienda?", review['voted_up'])
    print('-' * 40)

Usuario: 76561198079669556
Comentario: Para los que nunca lo han jugado y vinieron por recomendación o por el motivo que sea, no instalen esta m1erda de juego, por favor hagan caso a mi advertencia, tengo 10 años jugando esto, y dota ha arruinado mi vida, cuando creo que por fin dejé de jugarlo, algo o alguien o un pase de batalla, aparece de la nada y regresas por más, nunca es suficiente, nunca te llena, pierdes mas de lo que ganas y hay gente que merece algo peor que el infierno, este juego arruina la existencia misma, hay desesperación, agonía, desolación, peruanos, Techies, sufrimiento y un fuerte impulso por el su1c11dio, hermanos, por favor alejense de este juego y vivan sus vidas, enamorense, tengan hijos, viajen, coman, F0ll3n, vivan la vida, disfruten lo que nunca pude disfrutar, aun hay tiempo para ustedes, ya es tarde para mi.
¿Recomienda? False
----------------------------------------
Usuario: 76561198288933432
Comentario: Un juego con excelsas mecanicas, pero con la comun

# Funciones para preprocesamiento de texto y lematización

In [5]:
import re, string

nlp=spacy.load('es_core_news_md')

stop_words = ['el', 'la', 'lo', 'los', 'las', 'un', 'una', 'unos', 'unas', 'me', 'a', 'de', 'se', 'te']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #selecciona símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)] #obvia stop_words y después quita signos de puntuación
    filtered_text = ' '.join(filtered_tokens)

    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text

In [6]:
reviews = [review['review'] for review in data['reviews'] ]                # Obtener reviews a partir del diccionario
ratings  = [review['voted_up'] for review in data['reviews'] ]             # Obterner ratings asociados a las reviews anteriores

dataframe = pd.DataFrame({'reviews' : reviews , 'ratings' : ratings })     # Transformamos a formato pd.DataFrama para aprovechar sus múltiples utilidades

El primer paso del preprocesado va a ser eliminar las reseñas que no están en español. (En algunos juegos el filtrado de la API es imperfecto y algunas reseñas se muestran en inglés). Para esto nos haremos servir de la librería langdetect.

In [7]:
from langdetect import detect

dataframe['language'] = dataframe['reviews'].apply(detect)      # Generamos una columna adicional que guarda el idioma de la reseña
dataframe = dataframe[  dataframe['language'] == 'es'  ]        # Guardamos únicamente las entradas en español   
dataframe.head()

,reviews,ratings,language
0,Para los que nunca lo han jugado y vinieron po...,False,es
1,"Un juego con excelsas mecanicas, pero con la c...",False,es
2,"Es muy entretenido, si tienes horas libres pue...",True,es
3,"La verdad esque es muy buen juego, lo actualiz...",True,es
4,"después de 3 años siguiendo jugando, puedo dec...",True,es


El siguiente paso, muy necesario en nuestro contexto, es la corrección ortográfica: las reseñas de Steam muchas veces contienen _slangs_, insultos y un tono informal, lo cual dificulta cualquier tipo de clasificación. En este caso, haremos uso de la librería **spellchecker**:

In [8]:
from spellchecker import SpellChecker

spell = SpellChecker( language='es')
dataframe['reviews_c'] = dataframe['reviews'].apply(spell.correction)

# Obtenemos cuales son los juegos de Steam más jugados

Realizamos Web Scraping en la página de Steam, portal de venta de videojuegos, para obtener cuales son los juegos más jugados. De esta forma nos aseguramos que tenemos gran cantidad de comentarios en todos los juegos.

Para realizar el Web Scraping utilizamos Selenium ya que necesitamos interactuar con la página de Steam. Concretamente necesitamos scrollear para alcanzar el final de las página y así que cargue más juegos.

In [9]:
# Necesitamos selenium para hacer scroll en la página y cargar elementos

n = 9 # Número de scrolls
      # La página empieza con 50 juegos y carga 50 más por scroll

# Configurar Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializar driver
driver = webdriver.Chrome(options=chrome_options)

# Cargar la página
driver.get("https://store.steampowered.com/search/?untags=9130&filter=topsellers&ndl=1")

# Realizar scrolls
for _ in range(n):
    # Scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar a que cargue contenido nuevo
    time.sleep(2)

# Obtener el HTML después de todos los scrolls
html = driver.page_source

# Cerrar el navegador
driver.quit()

In [10]:
soup = BeautifulSoup(html, "html.parser")

tabla = soup.find_all("div", id="search_resultsRows")[0]

# Generamos una lista de diccionarios con la id y el título del juego.

# NOTA: Algunos de los "juegos más vendidos" no son juegos, como Steam Deck.
# Se puede filtrar más adelante, ya que en
# https://store.steampowered.com/api/appdetails?appids=1675200
# aparece como "type": "hardware", en vez de "type": "game" o "type": "dlc".
juegos = []

for game in tabla.find_all("a"):
  juegos.append({'id': game['data-ds-appid'], 'title': game.find("span", class_="title").text})

In [11]:
len(juegos)

500

In [12]:
juegos[:20]

[{'id': '1675200', 'title': 'Steam Deck'},
 {'id': '730', 'title': 'Counter-Strike 2'},
 {'id': '3017860', 'title': 'DOOM: The Dark Ages'},
 {'id': '3164500', 'title': 'Schedule I'},
 {'id': '1903340', 'title': 'Clair Obscur: Expedition 33'},
 {'id': '281990', 'title': 'Stellaris'},
 {'id': '1085660', 'title': 'Destiny 2'},
 {'id': '236390', 'title': 'War Thunder'},
 {'id': '1172710', 'title': 'Dune: Awakening'},
 {'id': '2669320', 'title': 'EA SPORTS FC™ 25'},
 {'id': '2623190', 'title': 'The Elder Scrolls IV: Oblivion Remastered'},
 {'id': '230410', 'title': 'Warframe'},
 {'id': '2488370', 'title': 'Cash Cleaner Simulator'},
 {'id': '3241660', 'title': 'R.E.P.O.'},
 {'id': '2767030', 'title': 'Marvel Rivals'},
 {'id': '1172470', 'title': 'Apex Legends™'},
 {'id': '1128920', 'title': 'EVERSPACE™ 2'},
 {'id': '381210', 'title': 'Dead by Daylight'},
 {'id': '1158310', 'title': 'Crusader Kings III'},
 {'id': '2967990', 'title': 'Train Sim World® 5'}]